## Установка необходимых библиотек

In [63]:
#!g1.1
%pip install transformers

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [64]:
#!g1.1
%pip install datasets

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


## Загрузка модели и токенайзера

In [63]:
#!g1.1
from transformers import AutoTokenizer,AutoModelForCausalLM
import torch
DEVICE = torch.device("cuda:0")

model_name = "sberbank-ai/rugpt3medium_based_on_gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name, bos_token="[SJ]", eos_token ="[EJ]", pad_token = '<pad>')
model = AutoModelForCausalLM.from_pretrained(model_name).to(DEVICE)
model.resize_token_embeddings(len(tokenizer))
tokenizer.save_pretrained("tokenizer.json")

Downloading:   0%|          | 0.00/674 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.54M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/1.61G [00:00<?, ?B/s]

('tokenizer.json/tokenizer_config.json',
 'tokenizer.json/special_tokens_map.json',
 'tokenizer.json/vocab.json',
 'tokenizer.json/merges.txt',
 'tokenizer.json/added_tokens.json',
 'tokenizer.json/tokenizer.json')

In [64]:
#!g1.1
torch.cuda.is_available()

True

## Подготовка датасета

In [65]:
#!g1.1
# Датасет взят отсюда https://www.kaggle.com/datasets/nikuson/ru-jokes-about-stirlitz
train_path = 'train_dataset.txt'
with open("Stirlitz.txt", encoding="utf-8") as input_file, open(train_path, "w", encoding="utf-8") as out_file:
    for line in input_file.read().split("* * *"):
        if line != "" and line != "\n" and line != " ":
            line = line.replace('\n\n', ' ')
            out_file.write(f'Шутка: {line.lstrip().rstrip()} <|end|>\n')

In [66]:
#!g1.1
from transformers import DataCollatorForLanguageModeling
from datasets import Dataset


train_dataset = Dataset.from_text(train_path)
train_dataset = train_dataset.map(lambda examples: tokenizer(examples["text"]),remove_columns=['text'])
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])


data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Using custom data configuration default-e440a7a8809ce5ab
/home/jupyter/.local/lib/python3.8/site-packages/datasets/builder.py:712: FutureWarning: 'use_auth_token' was deprecated in version 2.7.1 and will be removed in 3.0.0. Pass `use_auth_token` to the initializer/`load_dataset_builder` instead.
  warnings.warn(


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /tmp/xdg_cache/huggingface/datasets/text/default-e440a7a8809ce5ab/0.0.0. Subsequent calls will reuse this data.


Failed to deserialize variable 'train_dataset'. Run the following code to delete it:
  del_datasphere_variables('train_dataset')
Traceback (most recent call last):
  File "/kernel/lib/python3.8/site-packages/ml_kernel/state/state_protocol.py", line 283, in _load_component
    value = unpickler.load()
  File "/home/jupyter/.local/lib/python3.8/site-packages/datasets/table.py", line 1075, in __setstate__
    table = _memory_mapped_arrow_table_from_file(path)
  File "/home/jupyter/.local/lib/python3.8/site-packages/datasets/table.py", line 50, in _memory_mapped_arrow_table_from_file
    memory_mapped_stream = pa.memory_map(filename)
  File "pyarrow/io.pxi", line 950, in pyarrow.lib.memory_map
  File "pyarrow/io.pxi", line 911, in pyarrow.lib.MemoryMappedFile._open
  File "pyarrow/error.pxi", line 144, in pyarrow.lib.pyarrow_internal_check_status
  File "pyarrow/error.pxi", line 113, in pyarrow.lib.check_status
FileNotFoundError: [Errno 2] Failed to open local file '/tmp/xdg_cache/huggingf

  0%|          | 0/296 [00:00<?, ?ex/s]

Failed to deserialize variable 'train_dataset'. Run the following code to delete it:
  del_datasphere_variables('train_dataset')
Traceback (most recent call last):
  File "/kernel/lib/python3.8/site-packages/ml_kernel/state/state_protocol.py", line 283, in _load_component
    value = unpickler.load()
  File "/home/jupyter/.local/lib/python3.8/site-packages/datasets/table.py", line 1075, in __setstate__
    table = _memory_mapped_arrow_table_from_file(path)
  File "/home/jupyter/.local/lib/python3.8/site-packages/datasets/table.py", line 50, in _memory_mapped_arrow_table_from_file
    memory_mapped_stream = pa.memory_map(filename)
  File "pyarrow/io.pxi", line 950, in pyarrow.lib.memory_map
  File "pyarrow/io.pxi", line 911, in pyarrow.lib.MemoryMappedFile._open
  File "pyarrow/error.pxi", line 144, in pyarrow.lib.pyarrow_internal_check_status
  File "pyarrow/error.pxi", line 113, in pyarrow.lib.check_status
FileNotFoundError: [Errno 2] Failed to open local file '/tmp/xdg_cache/huggingf

## Аргументы для обучения

In [67]:
#!g1.1
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="StirlitzJokeOnRuGPT2_Medium",
    overwrite_output_dir=True,
    num_train_epochs=50,
    logging_steps=100,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=8,
    warmup_steps=10,
    save_strategy = "steps",
    save_steps = 200,
    weight_decay=0.05
)
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset, 
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None)
) 

Failed to deserialize variable 'train_dataset'. Run the following code to delete it:
  del_datasphere_variables('train_dataset')
Traceback (most recent call last):
  File "/kernel/lib/python3.8/site-packages/ml_kernel/state/state_protocol.py", line 283, in _load_component
    value = unpickler.load()
  File "/home/jupyter/.local/lib/python3.8/site-packages/datasets/table.py", line 1075, in __setstate__
    table = _memory_mapped_arrow_table_from_file(path)
  File "/home/jupyter/.local/lib/python3.8/site-packages/datasets/table.py", line 50, in _memory_mapped_arrow_table_from_file
    memory_mapped_stream = pa.memory_map(filename)
  File "pyarrow/io.pxi", line 950, in pyarrow.lib.memory_map
  File "pyarrow/io.pxi", line 911, in pyarrow.lib.MemoryMappedFile._open
  File "pyarrow/error.pxi", line 144, in pyarrow.lib.pyarrow_internal_check_status
  File "pyarrow/error.pxi", line 113, in pyarrow.lib.check_status
FileNotFoundError: [Errno 2] Failed to open local file '/tmp/xdg_cache/huggingf

## Запуск обучения

In [ ]:
#!g1.1
trainer.train()

***** Running training *****
  Num examples = 296
  Num Epochs = 50
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 8
  Total optimization steps = 200


Step,Training Loss
100,2.366400


Saving model checkpoint to StirlitzJokeOnRuGPT2_Medium/checkpoint-200
Configuration saved in StirlitzJokeOnRuGPT2_Medium/checkpoint-200/config.json
Model weights saved in StirlitzJokeOnRuGPT2_Medium/checkpoint-200/pytorch_model.bin


## Тестовый запуск (Необязательно)

In [ ]:
#!g1.1
text = "Шутка:"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=0.6,
                        top_p=1,
                        top_k=40,
                        max_length=300,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text[:generated_text.find('<|end|>')])